In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)
import dataset_comp
import snapshot
import curve_fit

In [ ]:
importlib.reload(dataset_comp)
importlib.reload(snapshot)
importlib.reload(curve_fit)

# Metallicity

Metallicity of an individual star is defined here as the mass fraction of the metals: 

$Z^*$ $ = \frac{ \sum_{j>\text{He}} m_j}{\sum_k m_k}$. 

Metallicity of a galaxy is defined as the mass weighted average of the star particles of that galaxy: 

$Z = \frac{ \sum_j m_j Z^*_i}{\sum_k m_k}$ (in the EAGLE simulations). 

I further normalize the metallicity of a galaxy by the solar metallicity $Z_\odot = 0.0134$ (Wikipedia), and use the log-metallicity of the normalized quantity: 

$Z = \log_{10} \frac{ \sum_j m_j Z^*_i}{\sum_k m_k} - \log_{10} Z_\odot$.

## Motivation

What makes metallicity in low-mass halos lower:
- Gas and dust less bound; some high-metallicity material gets ejected into the inter-galactic medium
- Formed earlier, from less recycled gas
- Lower recycling rate (less active?)

A low-mass galaxy creates a relatively shallow potential well for the inter-stellar gas and dust. Thus, high-metallicity material is more easily ejected out of a low-mass galaxy, in a supernova explosion. Therefore, metallicity will tend to be higher in more massive galaxies. 

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
data = {
    'plain-LCDM': {
        'snapshot': snapshot.Snapshot('V1_MR_fix', snap_id),
        'M31_identifier': (1, 0),
        'MW_identifier': (2, 0),
        'Color': ['black', 'gray']
    },
    'curv-p082': {
        'snapshot': snapshot.Snapshot('V1_MR_curvaton_p082_fix', snap_id),
        'M31_identifier': (1, 0),
        'MW_identifier': (1, 1),
        'Color': ['red', 'pink']
    }
}

Set the low-mass threshold – subhalos, whose $v_\mathrm{max}$ falls below this (in km/s) are excluded as potentially non-physical:

In [ ]:
lowm = 10

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
metal_sun = np.log(0.0134)

for name, sim_data in data.items():
    # Get data:\frac{ \sum_j m_j Z^*_i}{\sum_k m_k}
    snap = sim_data["snapshot"]
    sm = snap.get_subhalos("Stars/Mass") * units.g.to(units.Msun)
    metal = np.log(snap.get_subhalos("Stars/Metallicity")) - metal_sun
    
    # Split into satellites:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"]
    )
        
    # Set low mass limit for physical halos:
    mask_vmax = dataset_comp.prune_vmax(snap, low_lim=10)
    
    # Select only luminous:
    mask_lum,_ = dataset_comp.split_luminous(snap)
    mask_m31 = mask_m31[mask_lum]
    mask_mw = mask_mw[mask_lum]
    mask_isol = mask_isol[mask_lum]
    mask_vmax = mask_vmax[mask_lum]
    
    # Add datasets to dictionary:
    data[name]['SM'] = sm[mask_lum]
    data[name]['Metallicity'] = metal[mask_lum]

    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': np.logical_or(mask_m31, mask_mw),
        'Isolated': mask_isol,
        'Physical': mask_vmax
    }

## Plot

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 12,
              'axes.labelsize': 10,
              'xtick.labelsize': 9,
              'ytick.labelsize': 9,
              'legend.fontsize': 10}

# Marker size
ms = 15
a = 0.7

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(sharey=True, figsize=(3, 3))

ax.set_xscale('log')
ax.set_xlim(10**6, 5 * 10**9)
ax.set_ylim(-4, 1.5)    
ax.set_xlabel('$M_*[\mathrm{M_\odot}]$')
ax.set_ylabel(r'$\langle Z \rangle$')

# Add scatter plots:
for i, (name, sim_data) in enumerate(data.items()):
    mask = np.logical_and(sim_data['Selections']['Satellite'],
                          sim_data['Selections']['Physical'])
    x = sim_data['SM'][mask]
    y = sim_data['Metallicity'][mask]
    
    ax.scatter(x, y, alpha=a, marker='+',
               c=sim_data["Color"][0], s=ms)
    
    mask = np.logical_and(sim_data['Selections']['Isolated'],
                          sim_data['Selections']['Physical'])
    x = sim_data['SM'][mask]
    y = sim_data['Metallicity'][mask]
    ax.scatter(x, y, alpha=a, marker='s',
               facecolor='none', s=ms, edgecolor=sim_data["Color"][0])

# axes[0].legend(loc='lower right')

In [ ]:
dummy_plots = []
for sim_name, sim_data in data.items():
    splot = ax.scatter([], [], c=sim_data["Color"][0], alpha=a, marker='o', s=ms)
    dummy_plots.append(splot)
    
sim_legend = ax.legend(dummy_plots, list(data.keys()), loc='upper right')
ax.add_artist(sim_legend)

ax.axvline(10**7, c='gray', linestyle="dotted")

ax.scatter([], [], c=data["plain-LCDM"]["Color"][0], alpha=a, marker='+',
           s=ms, label="Satellite")
ax.scatter([], [], edgecolor=data["plain-LCDM"]["Color"][0], alpha=a, marker='s',
           facecolor='none', s=ms, label="Isolated")
ax.legend(loc='lower right')

fig

### Save the Figure

In [ ]:
# Construct saving location:
filename = 'metallicity.png'

home = os.path.abspath(os.path.join('..'))
path = os.path.join(home,'Figures', 'MediumResolution')
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')